# Лабораторная работа 9.
# Создание текстового файла большого объема с помощью генераторной функции


Трофимова Вероника Сергеевна, 06.06.2025

In [5]:
from random import randint

In [7]:
?randint

Signature: randint(a, b)
Docstring:
Return random integer in range [a, b], including both end points.
        
File:      c:\users\veronika\anaconda3\lib\random.py
Type:      method

In [9]:
latin_code = (ord('a'), ord('z')) # соответствует type_symbols = 'latin'
f'Код символа: {(c:=randint(*latin_code))}, символ: {chr(c)}'

'Код символа: 108, символ: l'

In [11]:
from time import process_time

In [13]:
?process_time

Docstring:
process_time() -> float

Process time for profiling: sum of the kernel and user-space CPU time.
Type:      builtin_function_or_method

In [15]:
start = process_time()
[x**2 for x in range(1_000_000)]
end = process_time()
print(f'{end - start} секунд')

0.09375 секунд


## Задание 9.1. Запись данных в текстовый файл


In [18]:
file_name = 'test.txt'
f = open(file_name, 'w')

In [20]:
lines_list = [f'{x}\n' for x in 'test']
lines_list

['t\n', 'e\n', 's\n', 't\n']

In [22]:
f.writelines(lines_list)

In [24]:
f.close()

In [26]:
with open(file_name,'r') as f:
    for line in f:
        print(line, end='')

t
e
s
t


In [28]:
lines_gen_expr = (f'{x}\n' for x in range(10**6))
def lines_gen_fun(number_lines=10**6):
 yield from (f'{x}\n' for x in range(number_lines))
with open(file_name,'w') as f:
 f.writelines(lines_gen_expr)
 f.writelines(lines_gen_fun())

In [29]:
def lines_gen_fun(number_lines = 10**6):
    file_size = 0
    for x in range(number_lines):
        line = f'{x}\n'
        yield line
        file_size += len(line)
    print(f'{file_size/1024**2} Mb')

with open(file_name,'w') as f:
    f.writelines(lines_gen_fun())

6.569757461547852 Mb


In [32]:
len('f'), len('f'.encode('utf8')), len('ц'), len('ц'.encode('utf8'))

(1, 1, 1, 2)

In [34]:
with open(file_name,'w') as f:
    start = process_time()
    f.writelines(lines_gen_fun()) 
    end = process_time()
    print(end - start, 'секунд')

6.569757461547852 Mb
0.875 секунд


In [36]:
def lines_gen_fun(number_lines=10**6, status=False):
    """
    Генераторная функция для создания строк с числами от 0 до number_lines.
    Резултат это статус выполнения при включеном параметре `status`.
    """
     
    file_size = 0    # Общий размер файла 
    for x in range(number_lines):
        line = f'{x}\n'  #Cтрока это число + перенос строки
        yield line  #Возвращаем сгенеренную строку
        file_size += len(line) #Увеличиваем размер на длину текущей строки
        if status and x % 10 == 0:  #Выводим прогресс(поставила ограничение на вывод каждой 10 строки, т.к комп отказывается по другому работать:( )
            status_number = x / number_lines * 100       # Процент выполнения
            print(f'\r{int(status_number)}%', end='', flush=True)  # Выводим процент выполнения с перезапианной строкой 
    print(f'\n {file_size / 1024**2} Mb')  

In [38]:
with open(file_name,'w') as f:
    start = process_time()
    f.writelines(lines_gen_fun(3*10**3, status=True))
    end = process_time()
print(f'{end - start} секунд')

99%
 0.013246536254882812 Mb
0.15625 секунд


## Задание 9.2. Генерация строк

In [40]:
latin_code = (ord('a'), ord('z')) 
number_words = (10, 20)
number_symbols = (5, 10)

In [42]:
len_word = randint(*number_symbols)
word = ''.join([chr(randint(*latin_code)) for i in range(len_word)])
word

'jgkbuow'

In [58]:
import random
def random_string():
    alphabet_lat = [chr(i) for i in range(65, 91)]       
    alphabet_kir = [chr(i) for i in range(1040, 1072)]   
    alphabet_kir.append(chr(1025))                       
    digits = [chr(i) for i in range(48, 58)]     
    
    quan_words = random.randint(1, 10)  
    words = []
    
    for _ in range(quan_words):
        word_length = random.randint(1, 15)  
  
        selection = random.choice([alphabet_lat, digits, alphabet_kir])
        word = ''.join(random.choice(selection) for _ in range(word_length))
        words.append(word)
    
    return ' '.join(words)

print(random_string())

5347471 570024906176731 2217611156 ODJXTIAKMELBKEN ЫЦЭШПОТЪОЭЩГВ ИШФСДЫЩОС U РМБЕЦЬЫФЖЙ


In [64]:
def get_symbol_set(type_symbols):   #Отдельно выношу функцию для определения букв и цифр
    latin_upper = [chr(i) for i in range(65, 91)]   
    latin_lower = [chr(i) for i in range(97, 123)]  
    latin_full = latin_upper + latin_lower

    cyrillic_upper = [chr(i) for i in range(1040, 1072)] + [chr(1025)]  
    cyrillic_lower = [chr(i) for i in range(1072, 1104)] + [chr(1105)] 
    cyrillic_full = cyrillic_upper + cyrillic_lower

    digits = [chr(i) for i in range(48, 58)] 

    symbol_sets = {
        'latin': latin_full,
        'digits': digits,
        'kiril': cyrillic_full,
        'mix': latin_full + cyrillic_full + digits
    }

    result = symbol_sets.get(type_symbols)
    if result is None:
        raise ValueError("Правильно введите тип символов")
    return result

In [66]:
def lines_gen_fun_v2(file_size, type_symbols, number_words, number_symbols):
    """
    Генераторная функция для создания строк с случайными символами.
    Строки генерируются до достижения указанного размера `file_size`
    """
    
    bytes_limit = file_size * 1024**2  #Мегабайты -> байты
    symbol_set = get_symbol_set(type_symbols)  
    total_size = 0  #Для подсчёта текущего размера файла

    while total_size < bytes_limit:   # Генерируем строки, пока их общий размер не достигнет `bytes_limit`
        my_list = [''.join([random.choice(symbol_set) for _ in range(number_symbols)]) for _ in range(number_words)] #Создание случайных слов
        line = ' '.join(my_list)  #Объединяем слова в одну строку 
        
        line_size = len(line.encode('utf-8'))   #Вычисляем размер строк

        if total_size + line_size > bytes_limit:  # Если строка превышает лимит останавливаем
            break 

        yield line  
        total_size += line_size   

In [68]:
file_size = 1 
type_symbols = 'mix'
number_words = 10
number_symbols = 10
      
generator = lines_gen_fun_v2(file_size, type_symbols, number_words, number_symbols)

In [70]:
print(next(generator))

zОgshКмл9о BМl4ъHxЮXА сОЖхFыqыQс dЦЧfщJё4ьф зэВkПjtFhм CuЖбNnыЗhц trаУцVOЦNw ZQxlэxHфbй АbущgUчdыЧ ЩNХ1ЦКСfуs


In [72]:
for i in range(10):
    print(next(generator))

SЖрMm37нp4 SnkьйБ3йюx ИcDTвFМoAД NЪ4иБED7sg ЪжвNыЯЦе3P yеSь6яжНOт lтOqpюфNЙЪ aвhГ6ЩMJВx оТАфнvфЮси dхЬЖсyзв4h
УZыНЕXУупд WяИyШрVGуу ТbУkPЕWс66 hИзЪрenУЗr fТж7сBJЪxr хтbyЫ9ПИRН чТzcYЭьjк3 9ШЮjS0юМch э18ГGЯpyНp 3GXfырmA2Ж
knUыXзкэпL ZИraэдOYIn ЧnhhKНnАlz ЛЧОVЩRЪEVФ мqЁнsлlД8о К4йС7ЙАъып ШEzyиyоbеЗ qNyхгжуёjЭ 3SAyй42Gщ1 xЙВеRvBVЭС
aШДте4ArRэ GhВЛYы4CвG YNАРъЛЛyЪа юкзtчhЫИGO uНWпЭAЭЖZТ Цi3NмРtpgИ ПoНcДhkw2h ь2mЗRKЗKЭЯ 1sыbkGрOц4 9oВdyq4ББ6
ЙьtЙБШHIгh ЪNеNEНgeuj тМkБPOSиЦл ШWxжd2PTGХ гQЭМтЛНYPч yсaBгсшvzЗ akdЛКkZ9жи ьБiьaрЦ9Н1 mУ5ТqАТbЪф ХшбrЬ2lQЙG
FMъLdЙhМXB fEСwхГрОCД 2нKiъшПуEA Gbи6ЬМoхЪq сeч368mРщш ClгЛФяuЮЖю FgsNоk9ЧкЮ nсkПзпAuз0 4МBгzbТЯЁy И2LИx4Ayсr
Nшяй43PдXй ЫУQly7ZлQf nJсZьQшceк 8QонAр7ТPх иteмNDъ8сэ раNuеsцВыГ МЮвeбJ4dЪе иlёzBШГпОU Тштmмm8JCY RьMqm3ёОgУ
эЗтKЮrУEПA XhщюIСЮР6Н зhСDчHmмeu ОШn9ПнАufп FссЗэрКUcl JVkТЦМwмпи Бp5ФjАвщ6ю rI0пЗKuCqщ cфпй3ЪX7DY 3fгМWAUв6U
AGоRpmMyВH юиFpqМЕJхp К5Л6яРоиsг аq7VюЁшdБё WNЫnсvёTPr нщT7тlкqШе ПКOpАMЩКXг QЖхр5LQiнИ uCX2МeЮPД6 И33ЛыЛrEжS
ЖиG2gфВm0n

## Задание 9.3. Запись сгенерированных строк в текстовый файл


In [74]:
import time

def file_gen(file_name: str,
             file_size: int,
             type_symbols: str = 'latin',
             number_words: (int, int) = (10, 20),
             number_symbols: (int, int) = (5, 10)):
    """
    Генерирует файл со случайными строками.
    
    """
    
    start_time = time.process_time() # Таймер чтобы времени выполнения
    bytes_written = 0  # Подсчет записанных байтов
    bytes_limit = file_size * 1024**2   #Мегабайты -> байты
    step_size = bytes_limit // 10  #Шаг для обновления процента выполнения
    next_status = step_size  # Следующий порог обновления статуса

    with open(file_name, 'w') as f:
        for line in lines_gen_fun_v2(file_size, type_symbols,  random.randint(*number_words), random.randint(*number_symbols)): # Случайное количество слов и длина слова
            f.write(line + '\n') # Добавляем строку в файл учитывая `\n`
            bytes_written += len(line.encode('utf-8')) + 1  

            if bytes_written >= next_status: # Если записанный объем данных достиг порога 
                progress = int(bytes_written / bytes_limit * 100) # Рассчитываем процент выполнения
                print(f"\rПроцент записанных данных: {progress}%", end="", flush=True)
                next_status += step_size  # Обновляем порог для следующего обновления статуса

    print(f"\nФактический размер: {bytes_written / 1024**2:.2f} МБ")
    print(f"Время выполнения: {time.process_time() - start_time:.2f} секунд")

In [76]:
file_name="result1.txt"
file_size=1
type_symbols="mix"
file_gen(file_name, file_size,type_symbols)

Процент записанных данных: 100%
Фактический размер: 1.01 МБ
Время выполнения: 0.38 секунд


In [78]:
file_name="result2.txt"
file_size=5
type_symbols="kiril"
file_gen(file_name, file_size,type_symbols)

Процент записанных данных: 100%
Фактический размер: 5.03 МБ
Время выполнения: 1.12 секунд


In [80]:
file_name="result3.txt"
file_size=10
type_symbols="latin"
file_gen(file_name, file_size,type_symbols)

Процент записанных данных: 100%
Фактический размер: 10.09 МБ
Время выполнения: 3.58 секунд
